https://excelsior-cjh.tistory.com/149

소프트맥스 회귀 - Softmax Regression

In [4]:
# softmax.py
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 상수 설정
DATA_DIR = '../data'
NUM_STEPS = 1000
MINIBATCH_SIZE = 200

# MNIST 데이터 불러오기
data = input_data.read_data_sets(DATA_DIR, one_hot=True)




Extracting ../data\train-images-idx3-ubyte.gz
Extracting ../data\train-labels-idx1-ubyte.gz
Extracting ../data\t10k-images-idx3-ubyte.gz
Extracting ../data\t10k-labels-idx1-ubyte.gz


In [6]:
#########
# 모델링 
#########
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
# b = tf.Variable(tf.random_uniform([10], -1., 1.))

# y_pred = tf.add(tf.matmul(x ,W), b)
y_pred = tf.matmul(x, W)
y_true = tf.placeholder(tf.float32, [None, 10])

# Cost(Loss) Function
cross_entropy = tf.reduce_mean(
                  tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y_true))

# Gradient Step(Optimze)
gd_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))




이미지(784 : 28 x 28) -> 숫자(10 : 1)

In [7]:
with tf.Session() as sess:
  # 학습
  sess.run(tf.global_variables_initializer())

  for step in range(NUM_STEPS):
      batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
      sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})

      if(step + 1) % 100 == 0:
          print(step+1, '|', sess.run(cross_entropy, feed_dict={x: batch_xs, y_true: batch_ys}))

  # 테스트
  ans = sess.run(accuracy, feed_dict={x: data.test.images,
                                     y_true: data.test.labels})

print("Accuracy: {:.2f}%".format(ans*100))



100 | 0.424384
200 | 0.22105129
300 | 0.35314646
400 | 0.3058225
500 | 0.40995637
600 | 0.3065945
700 | 0.25124827
800 | 0.27310926
900 | 0.249807
1000 | 0.2217989
Accuracy: 92.18%


kernel restarting...

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

def mnist_load():
  (train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

# Train set
# Image
train_x = train_x.reshape([train_x.shape[0], -1])
train_x = train_x.astype('float32') / 255
# Label
train_y = tf.keras.utils.to_categorical(train_y, num_classes=10)

# Test set
# Image
test_x = test_x.reshape([test_x.shape[0], -1])
test_x = test_x.astype('float32') / 255
# Label
test_y = tf.keras.utils.to_categorical(test_y, num_classes=10)

return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = mnist_load()

buffer_size = train_x.shape[0] + test_x.shape[0]

dataset = tf.data.Dataset.from_tensor_slices(({"image": train_x}, train_y))
dataset = dataset.shuffle(buffer_size).repeat().batch(MINIBATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))

y_pred = tf.matmul(x ,W)
y_true = tf.placeholder(tf.float32, [None, 10])

# Cost(Loss) Function
cross_entropy = tf.reduce_mean(
                  tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y_true))

# Gradient Step(Optimze)
gd_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

%%time
with tf.Session() as sess:
  # 학습
  sess.run(tf.global_variables_initializer())

  for step in range(NUM_STEPS):
      batch_xs, batch_ys = sess.run(next_batch)
      sess.run(gd_step, feed_dict={x: batch_xs['image'], y_true: batch_ys})

      if(step + 1) % 100 == 0:
          print(step+1, '|', sess.run(cross_entropy, feed_dict={x: batch_xs['image'], y_true: batch_ys}))

  # 테스트
  ans = sess.run(accuracy, feed_dict={x: test_x,
                                     y_true: test_y})
      
print("Accuracy: {:.2f}%".format(ans*100))










Extracting ../data\train-images-idx3-ubyte.gz
Extracting ../data\train-labels-idx1-ubyte.gz
Extracting ../data\t10k-images-idx3-ubyte.gz
Extracting ../data\t10k-labels-idx1-ubyte.gz


NameError: name 'train_x' is not defined